Docker Compose 起動エラー調査用ノートブック

このノートブックは各サービスごとに1セルを割り当て、各エンドポイントへ順にリクエストを送りステータスとレスポンスを確認します。

In [2]:
# --- ライブラリのインポート ---
import requests
import time
import traceback
import pandas as pd
from IPython.display import display
import docker
import subprocess
import logging
from pathlib import Path
from contextlib import contextmanager
from requests.exceptions import RequestException

# ロガー設定
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 設定
REQUEST_TIMEOUT = 5  # seconds
RETRY_COUNT = 1
RETRY_DELAY = 1

print("Imports done. REQUEST_TIMEOUT=", REQUEST_TIMEOUT)

Imports done. REQUEST_TIMEOUT= 5


In [10]:
# --- SERVICES 定義 ---
SERVICES = {
    "1-coordinator-ring": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "write": "/write",
            "read": "/read"
        },
        "test_data": {"key": "test_key", "value": "test_value"}
    },
    "2-quorum-consistency": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "write": "/write",
            "read": "/read"
        },
        "test_data": {"key": "test_key", "value": "test_value"}
    },
    "3-sharding-replica": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "write": "/write",
            "read": "/read"
        },
        "test_data": {"key": "test_key", "value": "test_value"}
    },
    "4-distributed-lock": {
        "port": 8000,
        "endpoints": {
            "health": "/stats",
            "acquire": "/acquire",
            "release": "/release"
        },
        "test_data": {"key": "test_resource", "owner": "test_client"}
    },
    "5-cache-aside": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "get": "/get",
            "set": "/set"
        },
        "test_data": {"entity_type": "user", "entity_id": "1", "data": {"name": "test"}}
    },
    "6-bloom-sstable": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "write": "/write",
            "read": "/read"
        },
        "test_data": {"key": "test_key", "value": "test_value"}
    },
    "7-rate-limiting": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "check": "/check_rate"
        },
        "test_data": {"user_id": "test_user"}
    },
    "8-line-streams": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "produce": "/produce",
            "consume": "/consume"
        },
        "test_data": {"message": "test_message", "consumer": "test_consumer"}
    },
    "9-session-store": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "login": "/login",
            "me": "/me"
        },
        "test_data": {"username": "test_user"}
    },
    "10-leaderboard": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "score": "/score",
            "top": "/top/10"
        },
        "test_data": {"user_id": "test_user", "score": 100}
    }
}

print("SERVICES configured with", len(SERVICES), "services")

SERVICES configured with 10 services


In [5]:
# --- ヘルパー関数: send_request ---

def try_parse_json(resp):
    try:
        return resp.json()
    except Exception:
        return None


def send_request(service_name, path, method='GET', json_body=None, params=None, timeout=REQUEST_TIMEOUT, session=None):
    cfg = SERVICES.get(service_name)
    port = cfg.get('port') if cfg else None
    url = f"http://localhost:{port}{path}"
    method = method.upper()
    last_exc = None

    for attempt in range(1, RETRY_COUNT+1):
        try:
            start = time.time()
            if session is not None:
                resp = session.request(method, url, json=json_body, params=params, timeout=timeout)
            else:
                resp = requests.request(method, url, json=json_body, params=params, timeout=timeout)
            elapsed = time.time() - start

            parsed = try_parse_json(resp)
            body = parsed if parsed is not None else resp.text

            return {
                'service': service_name,
                'url': url,
                'method': method,
                'status_code': resp.status_code,
                'body': body,
                'elapsed': elapsed,
                'error': None
            }
        except RequestException as e:
            last_exc = e
            time.sleep(RETRY_DELAY)
        except Exception as e:
            last_exc = e
            time.sleep(RETRY_DELAY)

    # all retries failed
    return {
        'service': service_name,
        'url': url,
        'method': method,
        'status_code': None,
        'body': None,
        'elapsed': None,
        'error': ''.join(traceback.format_exception_only(type(last_exc), last_exc)).strip()
    }

print('Helper functions defined')

Helper functions defined


In [ ]:
# --- 1-coordinator-ring のテスト ---
service = '1-coordinator-ring'
cfg = SERVICES[service]
print('Testing', service)

# health
r = send_request(service, cfg['endpoints']['health'], method='GET')
print(r)

# write
r_write = send_request(service, cfg['endpoints']['write'], method='POST', json_body=cfg['test_data'])
print(r_write)

# read (use params)
params = {'key': cfg['test_data'].get('key')}
r_read = send_request(service, cfg['endpoints']['read'], method='GET', params=params)
print(r_read)

In [ ]:
# --- 2-quorum-consistency のテスト ---
service = '2-quorum-consistency'
cfg = SERVICES[service]
print('Testing', service)

# health
r = send_request(service, cfg['endpoints']['health'], method='GET')
print(r)

# write
r_write = send_request(service, cfg['endpoints']['write'], method='POST', json_body=cfg['test_data'])
print(r_write)

# read (use params)
params = {'key': cfg['test_data'].get('key')}
r_read = send_request(service, cfg['endpoints']['read'], method='GET', params=params)
print(r_read)

In [ ]:
# --- 3-sharding-replica のテスト ---
service = '3-sharding-replica'
cfg = SERVICES[service]
print('Testing', service)

# health
r = send_request(service, cfg['endpoints']['health'], method='GET')
print(r)

# write
r_write = send_request(service, cfg['endpoints']['write'], method='POST', json_body=cfg['test_data'])
print(r_write)

# read (use params)
params = {'key': cfg['test_data'].get('key')}
r_read = send_request(service, cfg['endpoints']['read'], method='GET', params=params)
print(r_read)

In [ ]:
# --- 4-distributed-lock のテスト ---
service = '4-distributed-lock'
cfg = SERVICES[service]
print('Testing', service)

# health
r = send_request(service, cfg['endpoints']['health'], method='GET')
print(r)

# acquire
r_acq = send_request(service, cfg['endpoints']['acquire'], method='POST', json_body={"key": cfg['test_data']['key'], "owner": cfg['test_data']['owner']})
print(r_acq)

# release
r_rel = send_request(service, cfg['endpoints']['release'], method='POST', json_body={"key": cfg['test_data']['key'], "owner": cfg['test_data']['owner']})
print(r_rel)

In [8]:
# --- 5-cache-aside のテスト ---
service = '5-cache-aside'
cfg = SERVICES[service]
print('Testing', service)

# health
r = send_request(service, cfg['endpoints']['health'], method='GET')
print(r)

# set (Flaskのルーティングに合わせて修正)
r_set = send_request(
    service,
    f"/set/{cfg['test_data']['entity_type']}/{cfg['test_data']['entity_id']}",
    method='POST',
    json_body=cfg['test_data']['data']
)
print(r_set)

# get (Flaskのルーティングに合わせて修正)
r_get = send_request(
    service,
    f"/get/{cfg['test_data']['entity_type']}/{cfg['test_data']['entity_id']}",
    method='GET'
)
print(r_get)

# basic assertion
if r_get.get('body') is not None:
    if isinstance(r_get['body'], dict) and r_get['body'].get('data') == cfg['test_data']['data']:
        print('GET returned expected data')
    else:
        print('GET did not return expected data:', r_get['body'])

Testing 5-cache-aside
{'service': '5-cache-aside', 'url': 'http://localhost:8000/health', 'method': 'GET', 'status_code': 200, 'body': {'status': 'ok'}, 'elapsed': 0.0162200927734375, 'error': None}
{'service': '5-cache-aside', 'url': 'http://localhost:8000/set/user/1', 'method': 'POST', 'status_code': 200, 'body': {'cache_updated': True, 'status': 'success'}, 'elapsed': 0.01452493667602539, 'error': None}
{'service': '5-cache-aside', 'url': 'http://localhost:8000/get/user/1', 'method': 'GET', 'status_code': 200, 'body': {'data': {'name': 'test'}, 'source': 'cache', 'timestamp': '2025-09-10T23:18:27.972303'}, 'elapsed': 0.004725933074951172, 'error': None}
GET returned expected data


In [15]:
# --- 6-bloom-sstable のテスト ---
service = '6-bloom-sstable'
cfg = SERVICES[service]
print('Testing', service)

# health
r = send_request(service, cfg['endpoints']['health'], method='GET')
print(r)

# write
r_write = send_request(service, cfg['endpoints']['write'], method='POST', json_body=cfg['test_data'])
print(r_write)

# read (use params)
params = {'key': cfg['test_data'].get('key')}
r_read = send_request(service, cfg['endpoints']['read'], method='GET', params=params)
print(r_read)

Testing 6-bloom-sstable
{'service': '6-bloom-sstable', 'url': 'http://localhost:8000/health', 'method': 'GET', 'status_code': 200, 'body': {'status': 'ok'}, 'elapsed': 0.01183009147644043, 'error': None}
{'service': '6-bloom-sstable', 'url': 'http://localhost:8000/write', 'method': 'POST', 'status_code': 200, 'body': {'key': 'test_key', 'status': 'ok', 'value': 'test_value'}, 'elapsed': 0.004232883453369141, 'error': None}
{'service': '6-bloom-sstable', 'url': 'http://localhost:8000/read', 'method': 'GET', 'status_code': 200, 'body': {'key': 'test_key', 'status': 'ok', 'value': 'test_value'}, 'elapsed': 0.005250215530395508, 'error': None}


In [ ]:
# --- 7-rate-limiting のテスト ---
service = '7-rate-limiting'
cfg = SERVICES[service]
print('Testing', service)

# health
r = send_request(service, cfg['endpoints']['health'], method='GET')
print(r)

# call check_rate multiple times
results = []
for i in range(5):
    r_check = send_request(service, cfg['endpoints']['check'], method='GET', params=cfg['test_data'])
    print(f'check attempt {i+1}:', r_check)
    results.append(r_check)
    time.sleep(0.2)

In [ ]:
# --- 8-line-streams のテスト ---
service = '8-line-streams'
cfg = SERVICES[service]
print('Testing', service)

# health
r = send_request(service, cfg['endpoints']['health'], method='GET')
print(r)

# produce
r_prod = send_request(service, cfg['endpoints']['produce'], method='POST', json_body=cfg['test_data'])
print(r_prod)

# consume
r_consume = send_request(service, cfg['endpoints']['consume'], method='GET', params={'consumer': cfg['test_data'].get('consumer')})
print(r_consume)

In [ ]:
# --- 9-session-store のテスト ---
service = '9-session-store'
cfg = SERVICES[service]
print('Testing', service)

session = requests.Session()

# health
r = send_request(service, cfg['endpoints']['health'], method='GET', session=session)
print(r)

# login
r_login = send_request(service, cfg['endpoints']['login'], method='POST', json_body=cfg['test_data'], session=session)
print(r_login)

# me
r_me = send_request(service, cfg['endpoints']['me'], method='GET', session=session)
print(r_me)

In [ ]:
# --- 10-leaderboard のテスト ---
service = '10-leaderboard'
cfg = SERVICES[service]
print('Testing', service)

# health
r = send_request(service, cfg['endpoints']['health'], method='GET')
print(r)

# score
r_score = send_request(service, cfg['endpoints']['score'], method='POST', json_body=cfg['test_data'])
print(r_score)

# top
r_top = send_request(service, cfg['endpoints']['top'], method='GET')
print(r_top)

# check if user in top
if isinstance(r_top.get('body'), list):
    found = any((item.get('user_id') == cfg['test_data']['user_id']) for item in r_top['body'])
    print('User present in top:', found)

実行方法:

- 必要なパッケージをインストール: pip install requests pandas docker
- Jupyter で開いて各セルを順に実行するか、一括で実行して結果を確認してください。
- docker-compose のサービスが別ポートにバインドされている場合は `SERVICES` の port を調整してください。

備考: タイムアウトや接続拒否の詳細はセル出力に表示されます。ネットワークやコンテナのログと合わせて調査してください。